# Walmart competition: store sales forecasting

Link to the competition page: https://www.kaggle.com/competitions/walmart-recruiting-store-sales-forecasting

The project's aim is to predict store sales usng historical sales data for 45 Walmart stores located in different regions. Each store contains many departments, and sales must be forecasted for each department in each store.

I have used a combination of three different models:
- hybrid machine learning model composed of linear regression and XGBoost
- exponential smoothing for seasonal time series
- seasonal and trend decomposition using loess

The separate predictions from the three models are averaged to create the final submission for the competition. The final scores are 2832 (private score) and 2725 (public score), and the position on the leaderboard is ~60.

In [1]:
import numpy as np 
import pandas as pd 
import math
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.forecasting.stl import STLForecast
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller

import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip', parse_dates=['Date'])
train = train.set_index(keys=train.Date)
train.drop(columns=['Date', 'IsHoliday'], inplace=True)
train.head()

,Store,Dept,Weekly_Sales
Date,,,
2010-02-05,1,1,24924.50
2010-02-12,1,1,46039.49
2010-02-19,1,1,41595.55
2010-02-26,1,1,19403.54
2010-03-05,1,1,21827.90


In [3]:
test = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip',  parse_dates=['Date'])
test = test.set_index(keys=test.Date)
test.drop(columns=['IsHoliday', 'Date'], inplace=True)
test.head()

,Store,Dept
Date,,
2012-11-02,1,1
2012-11-09,1,1
2012-11-16,1,1
2012-11-23,1,1
2012-11-30,1,1


## Hybrid model: linear regression + XGBoost

In [4]:
def forecasting(train, test):
    dataframe_collection = {}
    
    #create y dataset for each dept in eallach store
    for store_n in range(1, 46):
        print('Store n: ', store_n)
        for dept_n in range(1, 100):
            y_train = train[(train['Store'] == store_n) & (train['Dept'] == dept_n)]
            y_test = test[(test['Store'] == store_n) & (test['Dept'] == dept_n)]
            
            if y_train.Weekly_Sales.count() == 0:
                continue
            if y_test.Store.count() == 0:
                continue
            if y_train.Weekly_Sales.count() == 1:
                df = pd.DataFrame({'store': y_train.Store.iloc[0], 
                                   'dept_n':y_train.Dept.iloc[0], 
                                   'Weekly_Sales':[y_train.Weekly_Sales.iloc[0]]}, 
                                   index=[pd.to_datetime(y_test.index.values[0]).date()])
                dataframe_collection['{}_{}'.format(store_n, dept_n)] = df
                
            
            else:
            
                #create trend feature
                index = pd.concat([y_train, y_test])
                dp = DeterministicProcess(
                        index = index.index,
                        constant = True,
                        order = 1,
                        drop = True)

                X = dp.in_sample()
                X_train, X_test = X.loc[y_train.index, :], X.loc[y_test.index, :]


                #Fit trend model
                model = LinearRegression(fit_intercept=False)
                model.fit(X_train, y_train.Weekly_Sales)

                #Make predictions
                y_fit = pd.DataFrame(
                                    model.predict(X_train),
                                    index=y_train.index,
                                    columns=['Weekly_Sales']
                )

                y_pred = pd.DataFrame(
                                    model.predict(X_test),
                                    index=y_test.index,
                                    columns=['Weekly_Sales']
                )


                X_train['Month'] = X_train.index.month
                X_test['Month'] = X_test.index.month

                #Convert df to series (squeeze)
                y_fit = y_fit.squeeze()
                y_pred = y_pred.squeeze()
                y_train = y_train.squeeze()

                #Create residuals (collection of detrended series) from the train set
                y_resid = y_train.Weekly_Sales - y_fit

                #Train XGBoost on the residuals
                xgb = XGBRegressor()
                xgb.fit(X_train.Month, y_resid)

                # Add the predicted residuals onto the predicted trends
                y_fit_boosted = xgb.predict(X_train.Month) + y_fit
                y_pred_boosted = xgb.predict(X_test.Month) + y_pred

                if len(y_pred_boosted) == 1:
                    df = pd.DataFrame(y_pred_boosted, columns=['Weekly_Sales'], index=[pd.to_datetime(y_test.index.values[0])])   #.date()
                    df['store'] = store_n
                    df['dept_n'] = dept_n
                    dataframe_collection['{}_{}'.format(store_n, dept_n)] = df
                else:
                    df = pd.DataFrame(y_pred_boosted)
                    df['store'] = store_n
                    df['dept_n'] = dept_n
                    dataframe_collection['{}_{}'.format(store_n, dept_n)] = df

    return dataframe_collection

In [5]:
df_list_xgb = forecasting(train, test)

Store n:  1
Store n:  2
Store n:  3
Store n:  4
Store n:  5
Store n:  6
Store n:  7
Store n:  8
Store n:  9
Store n:  10
Store n:  11
Store n:  12
Store n:  13
Store n:  14
Store n:  15
Store n:  16
Store n:  17
Store n:  18
Store n:  19
Store n:  20
Store n:  21
Store n:  22
Store n:  23
Store n:  24
Store n:  25
Store n:  26
Store n:  27
Store n:  28
Store n:  29
Store n:  30
Store n:  31
Store n:  32
Store n:  33
Store n:  34
Store n:  35
Store n:  36
Store n:  37
Store n:  38
Store n:  39
Store n:  40
Store n:  41
Store n:  42
Store n:  43
Store n:  44
Store n:  45


In [6]:
#Make dataframe from list of df
final_xgb = pd.DataFrame()
for df in iter(df_list_xgb.values()):
    final_xgb = pd.concat([final_xgb, df], axis=0)
final_xgb

,Weekly_Sales,store,dept_n
2012-11-02,22379.795706,1,1
2012-11-09,22360.742528,1,1
2012-11-16,22341.689349,1,1
2012-11-23,22322.636170,1,1
2012-11-30,22303.582992,1,1
...,...,...,...
2013-06-28,1201.557617,45,98
2013-07-05,1102.484576,45,98
2013-07-12,1107.903654,45,98
2013-07-19,1113.322731,45,98


In [7]:
#prepare final_xgb
submission_xgb = final_xgb.copy()
submission_xgb['date'] = pd.to_datetime(submission_xgb.index).date
submission_xgb['Id'] = submission_xgb['store'].astype(str) + '_' + submission_xgb['dept_n'].astype(str) + '_' + submission_xgb['date'].astype(str)
submission_xgb = submission_xgb.reset_index().drop(columns=['index', 'store', 'dept_n', 'date'])
submission_xgb = submission_xgb[['Id', 'Weekly_Sales']]
#drop duplicates
duplicates = submission_xgb[submission_xgb.duplicated(subset='Id')]
submission_xgb = submission_xgb.drop_duplicates(subset='Id')

submission_xgb

,Id,Weekly_Sales
0,1_1_2012-11-02,22379.795706
1,1_1_2012-11-09,22360.742528
2,1_1_2012-11-16,22341.689349
3,1_1_2012-11-23,22322.636170
4,1_1_2012-11-30,22303.582992
...,...,...
114995,45_98_2013-06-28,1201.557617
114996,45_98_2013-07-05,1102.484576
114997,45_98_2013-07-12,1107.903654
114998,45_98_2013-07-19,1113.322731


Competition score for xgboost predictions only: 
- private: 3560
- public: 3449

Competiotion score for xgboost averaged with est for seasonal time series:
- private: 3349
- public: 3251

## Exponential smoothing (EST) for seasonal time series

In [8]:
def check_stationarity(ts):
    dftest = adfuller(ts)
    adf = dftest[0]
    pvalue = dftest[1]
    critical_value = dftest[4]['5%']
    if (pvalue < 0.05) and (adf < critical_value):
        return True
    else:
        return False


def forecasting_est(train, test):
    dataframe_collection = {}
    
    for store_n in range(1, 46):
        print('Store n: ', store_n)
        
        for dept_n in range(1, 100):
            y = train[(train['Store'] == store_n) & (train['Dept'] == dept_n)]
            y_test = test[(test['Store'] == store_n) & (test['Dept'] == dept_n)]
            
            
            if y_test.Store.count() == 0:
                continue
            elif y.Weekly_Sales.count() == 0:
                continue
            elif y.Weekly_Sales.count() <= 8:
                continue
            
            else:
                #stationarity clause
                if check_stationarity(y.Weekly_Sales) == False:
                    
                    y = y.asfreq('W-FRI')
                    y = y.fillna(method='ffill') 
                    if y.Weekly_Sales.count() <= 99:
                        continue
                    else:
                        est = ExponentialSmoothing(y.Weekly_Sales, trend=None, seasonal='add', seasonal_periods=52, freq='W-FRI')
                        model_fit = est.fit()
                        y_pred = model_fit.predict(start=y_test.index[0], end=y_test.index[-1])

                else:
                    continue
                
                if len(y_pred) == 1:
                    df = pd.DataFrame(y_pred, columns=['Weekly_Sales'], index=[pd.to_datetime(y_test.index.values[0])])   
                    df['store'] = store_n
                    df['dept_n'] = dept_n
                    dataframe_collection['{}_{}'.format(store_n, dept_n)] = df
                else:
                    df = pd.DataFrame(y_pred, columns=['Weekly_Sales'])
                    df['store'] = store_n
                    df['dept_n'] = dept_n
                    dataframe_collection['{}_{}'.format(store_n, dept_n)] = df

    return dataframe_collection

In [9]:
df_list_est = forecasting_est(train, test)

Store n:  1
Store n:  2
Store n:  3
Store n:  4
Store n:  5
Store n:  6
Store n:  7
Store n:  8
Store n:  9
Store n:  10
Store n:  11
Store n:  12
Store n:  13
Store n:  14
Store n:  15
Store n:  16
Store n:  17
Store n:  18
Store n:  19
Store n:  20
Store n:  21
Store n:  22
Store n:  23
Store n:  24
Store n:  25
Store n:  26
Store n:  27
Store n:  28
Store n:  29
Store n:  30
Store n:  31
Store n:  32
Store n:  33
Store n:  34
Store n:  35
Store n:  36
Store n:  37
Store n:  38
Store n:  39
Store n:  40
Store n:  41
Store n:  42
Store n:  43
Store n:  44
Store n:  45


In [10]:
final_est = pd.DataFrame()
for df in iter(df_list_est.values()):
    final_est = pd.concat([final_est, df], axis=0)
final_est

,Weekly_Sales,store,dept_n
2012-11-02,35580.617215,1,1
2012-11-09,19182.260692,1,1
2012-11-16,19286.659222,1,1
2012-11-23,19227.425388,1,1
2012-11-30,23105.181912,1,1
...,...,...,...
2013-06-28,1090.133667,45,98
2013-07-05,1065.208237,45,98
2013-07-12,1057.488113,45,98
2013-07-19,930.921742,45,98


In [11]:
#prepare final_est
submission_est = final_est.copy()
submission_est['date'] = pd.to_datetime(submission_est.index).date
submission_est['Id'] = submission_est['store'].astype(str) + '_' + submission_est['dept_n'].astype(str) + '_' + submission_est['date'].astype(str)
submission_est = submission_est.reset_index().drop(columns=['index', 'store', 'dept_n', 'date'])
submission_est = submission_est[['Id', 'Weekly_Sales']]
submission_est

,Id,Weekly_Sales
0,1_1_2012-11-02,35580.617215
1,1_1_2012-11-09,19182.260692
2,1_1_2012-11-16,19286.659222
3,1_1_2012-11-23,19227.425388
4,1_1_2012-11-30,23105.181912
...,...,...
32220,45_98_2013-06-28,1090.133667
32221,45_98_2013-07-05,1065.208237
32222,45_98_2013-07-12,1057.488113
32223,45_98_2013-07-19,930.921742


## STL

In [12]:
def forecasting_stl(train, test):
    dataframe_collection = {}
    
    for store_n in range(1, 46):
        print('Store n: ', store_n)
        
        for dept_n in range(1, 100):
            y = train[(train['Store'] == store_n) & (train['Dept'] == dept_n)]
            y_test = test[(test['Store'] == store_n) & (test['Dept'] == dept_n)]
                     
            if y_test.Store.count() == 0:
                continue
            if y_test.Store.count() <= 8:
                continue            
            elif y.Weekly_Sales.count() <= 1:
                continue
            
            elif y.Weekly_Sales.count() < y_test.Store.count()*2:
                continue
            
            else:
                y = y.asfreq('W-FRI')
                y = y.fillna(0) 
                stlf = STLForecast(y.Weekly_Sales, ARIMA, model_kwargs=dict(order=(1, 1, 0), trend="t"), period=52)
                stlf_res = stlf.fit()
                y_pred = stlf_res.forecast(len(y_test))
                
                if len(y_pred) == 1:
                    df = pd.DataFrame(y_pred, columns=['Weekly_Sales'], index=[pd.to_datetime(y_test.index.values[0])])   
                    df['store'] = store_n
                    df['dept_n'] = dept_n
                    dataframe_collection['{}_{}'.format(store_n, dept_n)] = df
                else:
                    df = pd.DataFrame(y_pred, columns=['Weekly_Sales'])
                    df['store'] = store_n
                    df['dept_n'] = dept_n
                    dataframe_collection['{}_{}'.format(store_n, dept_n)] = df

    return dataframe_collection

In [13]:
df_list_stl = forecasting_stl(train, test)

Store n:  1
Store n:  2
Store n:  3
Store n:  4
Store n:  5
Store n:  6
Store n:  7
Store n:  8
Store n:  9
Store n:  10
Store n:  11
Store n:  12
Store n:  13
Store n:  14
Store n:  15
Store n:  16
Store n:  17
Store n:  18
Store n:  19
Store n:  20
Store n:  21
Store n:  22
Store n:  23
Store n:  24
Store n:  25
Store n:  26
Store n:  27
Store n:  28
Store n:  29
Store n:  30
Store n:  31
Store n:  32
Store n:  33
Store n:  34
Store n:  35
Store n:  36
Store n:  37
Store n:  38
Store n:  39
Store n:  40
Store n:  41
Store n:  42
Store n:  43
Store n:  44
Store n:  45


In [14]:
final_stl = pd.DataFrame()
for df in iter(df_list_stl.values()):
    final_stl = pd.concat([final_stl, df], axis=0)
final_stl

,Weekly_Sales,store,dept_n
2012-11-02,40908.844557,1,1
2012-11-09,19704.788080,1,1
2012-11-16,20057.239236,1,1
2012-11-23,21908.057444,1,1
2012-11-30,26279.558188,1,1
...,...,...,...
2013-06-28,818.641840,45,98
2013-07-05,807.279931,45,98
2013-07-12,848.662996,45,98
2013-07-19,860.318268,45,98


In [15]:
#prepare final_stl
submission_stl = final_stl.copy()
submission_stl['date'] = pd.to_datetime(submission_stl.index).date
submission_stl['Id'] = submission_stl['store'].astype(str) + '_' + submission_stl['dept_n'].astype(str) + '_' + submission_stl['date'].astype(str)
submission_stl = submission_stl.reset_index().drop(columns=['index', 'store', 'dept_n', 'date'])
submission_stl = submission_stl[['Id', 'Weekly_Sales']]
submission_stl

,Id,Weekly_Sales
0,1_1_2012-11-02,40908.844557
1,1_1_2012-11-09,19704.788080
2,1_1_2012-11-16,20057.239236
3,1_1_2012-11-23,21908.057444
4,1_1_2012-11-30,26279.558188
...,...,...
112862,45_98_2013-06-28,818.641840
112863,45_98_2013-07-05,807.279931
112864,45_98_2013-07-12,848.662996
112865,45_98_2013-07-19,860.318268


## Merge together dataframes and calculate mean of weekly_sales

In [16]:
#add column to submission_xgb with weekly_sales from est
merged_df = submission_xgb.merge(submission_est[['Id', 'Weekly_Sales']], on='Id', how='left', suffixes=('_xgb', '_est'))
merged_df = merged_df.merge(submission_stl[['Id', 'Weekly_Sales']], on='Id', how='left')
merged_df

,Id,Weekly_Sales_xgb,Weekly_Sales_est,Weekly_Sales
0,1_1_2012-11-02,22379.795706,35580.617215,40908.844557
1,1_1_2012-11-09,22360.742528,19182.260692,19704.788080
2,1_1_2012-11-16,22341.689349,19286.659222,20057.239236
3,1_1_2012-11-23,22322.636170,19227.425388,21908.057444
4,1_1_2012-11-30,22303.582992,23105.181912,26279.558188
...,...,...,...,...
114995,45_98_2013-06-28,1201.557617,1090.133667,818.641840
114996,45_98_2013-07-05,1102.484576,1065.208237,807.279931
114997,45_98_2013-07-12,1107.903654,1057.488113,848.662996
114998,45_98_2013-07-19,1113.322731,930.921742,860.318268


In [17]:
#Create new column with mean of all other columns
columns = merged_df.columns[1:] # all columns except the first
print(columns)
merged_df['Weekly_Sales'] = merged_df[columns].mean(axis=1)
merged_df = merged_df[['Id', 'Weekly_Sales']]
merged_df

Index(['Weekly_Sales_xgb', 'Weekly_Sales_est', 'Weekly_Sales'], dtype='object')


,Id,Weekly_Sales
0,1_1_2012-11-02,32956.419159
1,1_1_2012-11-09,20415.930433
2,1_1_2012-11-16,20561.862602
3,1_1_2012-11-23,21152.706334
4,1_1_2012-11-30,23896.107697
...,...,...
114995,45_98_2013-06-28,1036.777708
114996,45_98_2013-07-05,991.657581
114997,45_98_2013-07-12,1004.684921
114998,45_98_2013-07-19,968.187580


## Prepare df for submission

In [18]:
sample = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip')
sample

,Id,Weekly_Sales
0,1_1_2012-11-02,0
1,1_1_2012-11-09,0
2,1_1_2012-11-16,0
3,1_1_2012-11-23,0
4,1_1_2012-11-30,0
...,...,...
115059,45_98_2013-06-28,0
115060,45_98_2013-07-05,0
115061,45_98_2013-07-12,0
115062,45_98_2013-07-19,0


In [19]:
submission = sample.merge(merged_df[['Id', 'Weekly_Sales']], on='Id', how='left', suffixes=('_sample', '_submission'))
submission['Weekly_Sales_submission'] = submission['Weekly_Sales_submission'].fillna(0)
submission = submission[['Id', 'Weekly_Sales_submission']]
submission = submission.rename(columns={'Weekly_Sales_submission':'Weekly_Sales'})
submission

,Id,Weekly_Sales
0,1_1_2012-11-02,32956.419159
1,1_1_2012-11-09,20415.930433
2,1_1_2012-11-16,20561.862602
3,1_1_2012-11-23,21152.706334
4,1_1_2012-11-30,23896.107697
...,...,...
115059,45_98_2013-06-28,1036.777708
115060,45_98_2013-07-05,991.657581
115061,45_98_2013-07-12,1004.684921
115062,45_98_2013-07-19,968.187580


In [20]:
submission.to_csv('submission_4.csv',index=False)